In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [3]:
import pandas as pd
train = pd.read_csv("../input/train.csv")

In [4]:
X_train = train.drop(["label"],axis=1)
y_train = train["label"]

In [5]:
X = []
for i in range(60000):
  X.append(X_train.iloc[i].values.reshape([28,28]))
  

In [6]:
import numpy as np
X = np.array(X)

In [7]:
X = X.reshape(60000,28,28,1)

In [8]:
test = pd.read_csv("../input/test.csv")

In [9]:
X_test = test.drop(["id"],axis=1)

In [10]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

Using TensorFlow backend.


In [11]:
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten , Conv2D

# BUILD CONVOLUTIONAL NEURAL NETWORKS
nets = 15
model = [0] *nets
for j in range(nets):
    model[j] = Sequential()

    model[j].add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(128, kernel_size = 4, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Flatten())
    model[j].add(Dropout(0.4))
    model[j].add(Dense(10, activation='softmax'))

    # COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [12]:
import tensorflow as tf
Y_train = tf.keras.utils.to_categorical(y_train, 10)

In [13]:
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
# DECREASE LEARNING RATE EACH EPOCH
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
# TRAIN NETWORKS
history = [0] * nets
epochs = 45
for j in range(nets):
    X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X, Y_train, test_size = 0.1)
    history[j] = model[j].fit_generator(datagen.flow(X_train2,Y_train2, batch_size=64),
        epochs = epochs, steps_per_epoch = X_train2.shape[0]//64,  
        validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
    print("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j+1,epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

CNN 1: Epochs=45, Train accuracy=0.92473, Validation accuracy=0.92800
CNN 2: Epochs=45, Train accuracy=0.92424, Validation accuracy=0.93000
CNN 3: Epochs=45, Train accuracy=0.92510, Validation accuracy=0.92933
CNN 4: Epochs=45, Train accuracy=0.92424, Validation accuracy=0.94033
CNN 5: Epochs=45, Train accuracy=0.92341, Validation accuracy=0.93433
CNN 6: Epochs=45, Train accuracy=0.92502, Validation accuracy=0.93400
CNN 7: Epochs=45, Train accuracy=0.92473, Validation accuracy=0.93283
CNN 8: Epochs=45, Train accuracy=0.92497, Validation accuracy=0.93067
CNN 9: Epochs=45, Train accuracy=0.92456, Validation accuracy=0.93517
CNN 10: Epochs=45, Train accuracy=0.92411, Validation accuracy=0.92983
CNN 11: Epochs=45, Train accuracy=0.92547, Validation accuracy=0.92867
CNN 12: Epochs=45, Train accuracy=0.92397, Validation accuracy=0.93150
CNN 13: Epochs=45, Train accuracy=0.92302, Validation accuracy=0.93700
CNN 14: Epochs=45, Train accuracy=0.92306, Validation accuracy=0.93767
CNN 15: Epochs=

In [19]:
Xt = []
for i in range(10000):
  Xt.append(X_test.iloc[i].values.reshape([28,28]))

In [23]:
Xt = np.array(Xt)


In [24]:
Xt = Xt.reshape(10000,28,28,1)

In [25]:
results = np.zeros( (Xt.shape[0],10) ) 
for j in range(nets):
    results = results + model[j].predict(Xt)
results = np.argmax(results,axis = 1)

In [36]:
len(results)

10000

In [38]:
re = results.reshape(10000,)

In [41]:
print(re.tolist())

[0, 1, 2, 2, 3, 6, 8, 6, 5, 0, 3, 4, 4, 6, 8, 5, 6, 3, 6, 4, 4, 4, 2, 1, 5, 7, 8, 4, 4, 1, 5, 7, 7, 8, 1, 0, 9, 8, 0, 8, 2, 0, 4, 6, 2, 0, 3, 3, 2, 3, 2, 4, 9, 3, 0, 9, 9, 4, 6, 0, 4, 5, 4, 6, 1, 1, 0, 9, 5, 2, 7, 3, 4, 6, 5, 7, 1, 6, 1, 4, 9, 8, 1, 2, 4, 8, 9, 4, 1, 6, 3, 6, 2, 2, 2, 6, 4, 7, 7, 3, 9, 0, 9, 2, 8, 2, 3, 8, 2, 7, 5, 5, 3, 2, 7, 5, 6, 2, 7, 4, 0, 5, 6, 4, 7, 0, 8, 5, 0, 3, 1, 7, 9, 4, 9, 6, 4, 4, 6, 4, 3, 3, 3, 2, 2, 6, 0, 0, 1, 3, 4, 3, 3, 1, 9, 3, 3, 3, 9, 5, 6, 7, 7, 3, 2, 4, 0, 8, 7, 2, 2, 8, 9, 0, 2, 4, 4, 5, 7, 9, 9, 1, 3, 9, 1, 5, 5, 6, 0, 7, 4, 9, 1, 6, 0, 0, 0, 4, 6, 9, 0, 4, 2, 5, 5, 8, 6, 2, 1, 9, 3, 4, 7, 1, 9, 5, 9, 0, 2, 8, 5, 7, 7, 3, 2, 4, 5, 7, 8, 1, 9, 5, 6, 2, 9, 7, 4, 0, 9, 2, 1, 5, 7, 7, 0, 2, 4, 5, 3, 3, 8, 1, 6, 4, 4, 8, 6, 9, 8, 3, 3, 2, 4, 2, 2, 9, 3, 9, 2, 8, 6, 8, 8, 2, 7, 2, 5, 5, 9, 5, 8, 6, 4, 8, 4, 8, 3, 1, 2, 2, 2, 1, 8, 1, 3, 3, 4, 1, 7, 0, 5, 2, 0, 7, 6, 2, 7, 4, 7, 7, 1, 6, 6, 0, 7, 8, 7, 0, 9, 2, 0, 2, 5, 0, 8, 5, 8, 7, 7, 6, 1, 2, 9, 